In [216]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from tqdm.notebook import tqdm



In [217]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn') 
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [218]:
t1=pd.read_csv('./data./rename/t1_JEJU_MERCHANT.csv',encoding='cp949')
df=t1.head()

### 이름으로만 데이터프레임 만들기  

In [219]:
df=pd.DataFrame(t1.loc[t1['LG_CAT_NM']=='요식/유흥']['MCT_NM'].unique())
df.rename(columns = {0:'MCT_NM'},inplace=True)
df

# t1.loc[t1['LG_CAT_NM']=='요식/유흥']['MCT_NM'].unique()

,MCT_NM
0,젠
1,3657
2,가치
3,곰막
4,너븐
5,당당
6,도렐
7,먹돌
8,빵귿
9,신관


### naver map url만 뽑기

In [45]:
path='C:\selenium\chromedriver.exe'
driver=webdriver.Chrome(path)

df['naver_keyword'] = '제주도' + "%20" + df['MCT_NM']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass
    

        
   


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# # URL이 수집되지 않은 데이터는 제거합니다.
# df = df.loc[~df['naver_map_url'].isnull()]

이번에 찾을 키워드 : 0 / 536 행 제주도%20젠
이번에 찾을 키워드 : 1 / 536 행 제주도%203657
이번에 찾을 키워드 : 2 / 536 행 제주도%20가치
이번에 찾을 키워드 : 3 / 536 행 제주도%20곰막
이번에 찾을 키워드 : 4 / 536 행 제주도%20너븐
이번에 찾을 키워드 : 5 / 536 행 제주도%20당당
이번에 찾을 키워드 : 6 / 536 행 제주도%20도렐
이번에 찾을 키워드 : 7 / 536 행 제주도%20먹돌
이번에 찾을 키워드 : 8 / 536 행 제주도%20빵귿
이번에 찾을 키워드 : 9 / 536 행 제주도%20신관
이번에 찾을 키워드 : 10 / 536 행 제주도%20아리
이번에 찾을 키워드 : 11 / 536 행 제주도%20엘픽
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 12 / 536 행 제주도%20연돈
이번에 찾을 키워드 : 13 / 536 행 제주도%20오라
이번에 찾을 키워드 : 14 / 536 행 제주도%20우무
이번에 찾을 키워드 : 15 / 536 행 제주도%20주비
이번에 찾을 키워드 : 16 / 536 행 제주도%20치꼬
이번에 찾을 키워드 : 17 / 536 행 제주도%20크림
이번에 찾을 키워드 : 18 / 536 행 제주도%20풍로
이번에 찾을 키워드 : 19 / 536 행 제주도%20피즈
이번에 찾을 키워드 : 20 / 536 행 제주도%20하찌
이번에 찾을 키워드 : 21 / 536 행 제주도%20해왓
이번에 찾을 키워드 : 22 / 536 행 제주도%

이번에 찾을 키워드 : 177 / 536 행 제주도%20벨아벨카페
이번에 찾을 키워드 : 178 / 536 행 제주도%20봉주르마담
이번에 찾을 키워드 : 179 / 536 행 제주도%20불특정식당
이번에 찾을 키워드 : 180 / 536 행 제주도%20브라보비치
이번에 찾을 키워드 : 181 / 536 행 제주도%20비스트로낭
이번에 찾을 키워드 : 182 / 536 행 제주도%20상상함덕점
이번에 찾을 키워드 : 183 / 536 행 제주도%20섭지코지로
이번에 찾을 키워드 : 184 / 536 행 제주도%20솔지노형점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 185 / 536 행 제주도%20숙성도중문
이번에 찾을 키워드 : 186 / 536 행 제주도%20스시마타네
이번에 찾을 키워드 : 187 / 536 행 제주도%20스피닝울프
이번에 찾을 키워드 : 188 / 536 행 제주도%20신미영이네
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 189 / 536 행 제주도%20신화돌담집
이번에 찾을 키워

이번에 찾을 키워드 : 279 / 536 행 제주도%20도토리키친본점
이번에 찾을 키워드 : 280 / 536 행 제주도%20돈사돈  월정점
이번에 찾을 키워드 : 281 / 536 행 제주도%20링코제주시청점
이번에 찾을 키워드 : 282 / 536 행 제주도%20마노카페하우스
이번에 찾을 키워드 : 283 / 536 행 제주도%20맥파이블루버드
이번에 찾을 키워드 : 284 / 536 행 제주도%20문개항아리조천
이번에 찾을 키워드 : 285 / 536 행 제주도%20버거요제주본점
이번에 찾을 키워드 : 286 / 536 행 제주도%20복자씨연탄구이
이번에 찾을 키워드 : 287 / 536 행 제주도%20산호풍경이호점
이번에 찾을 키워드 : 288 / 536 행 제주도%20소금바치순이네
이번에 찾을 키워드 : 289 / 536 행 제주도%20수애기베이커리
이번에 찾을 키워드 : 290 / 536 행 제주도%20순전이해녀김밥
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 291 / 536 행 제주도%20쌍둥이횟집본점
이번에 찾을 키워드 : 292 / 536 행 제주도%20아베베베이커리
이번에 찾을 키워드 : 293 / 536 행 제주도%20어린왕자감귤밭
이번에 찾을 키워드 : 294 / 536 행 제주도%20이스트포레스트
이번에 찾을 키워드 : 295 / 536 행 제주도%20일돈성산흑돼지
이번에 찾을 키워드 : 296 / 536 행 제주도%20자매국수삼도점
Message: no such element: Unable to locate

이번에 찾을 키워드 : 353 / 536 행 제주도%20주식회사 아라파파
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 354 / 536 행 제주도%20주식회사 옥만이네
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 355 / 536 행 제주도%20주식회사 우동카덴
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 356 / 536 행 제주도%20주식회사 제주사월
Message: no such element: Unable to locate element: {"method":"css sele

이번에 찾을 키워드 : 408 / 536 행 제주도%20주식회사 임팩트컴퍼니
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 409 / 536 행 제주도%20주왕산업개발 주식회사
이번에 찾을 키워드 : 410 / 536 행 제주도%20투썸플레이스 용두암점
이번에 찾을 키워드 : 411 / 536 행 제주도%20바다를본돼지(서귀포점)
이번에 찾을 키워드 : 412 / 536 행 제주도%20배스킨라빈스제주함덕점
이번에 찾을 키워드 : 413 / 536 행 제주도%20빽다방서귀포올레중앙점
이번에 찾을 키워드 : 414 / 536 행 제주도%20순천미향제주산방산본점
이번에 찾을 키워드 : 415 / 536 행 제주도%20에이팩토리베이커리카페
이번에 찾을 키워드 : 416 / 536 행 제주도%20엔제리너스 제주공항T/O
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 417 / 536 행 제주도%20제주공항그때그집직영점
Message: no such element: Unable to locate element: {"met

이번에 찾을 키워드 : 452 / 536 행 제주도%20투썸플레이스 곽지과물해변점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 453 / 536 행 제주도%20(주)제주국제컨벤션센터(던킨)
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 454 / 536 행 제주도%20온더스톤브런치카페제주성산점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 455 / 536 행 제주도%20주식회사 큰돈가 서쪽대표본점
Message: no such element: Unable to locate eleme

이번에 찾을 키워드 : 480 / 536 행 제주도%20주식회사 호반호텔앤리조트제주지점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 481 / 536 행 제주도%20(주) 신세계푸드  데블스도어 제주점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 482 / 536 행 제주도%20주식회사 유동커피 루아앤티그리지점
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 483 / 536 행 제주도%20(주)파리크라상파리바게뜨 제주공항점
Message: no such element: Unable 

이번에 찾을 키워드 : 508 / 536 행 제주도%20치토스
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 509 / 536 행 제주도%20주식회사 뮤직박스
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 510 / 536 행 제주도%20토끼
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=105.0.5195.127)

이번에 찾을 키워드 : 511 / 536 행 제주도%20보물섬
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#

In [52]:
df.head()

,MCT_NM,naver_keyword,naver_map_url,menu_list
0,젠,제주도%20젠,https://m.place.naver.com/restaurant/36333194/...,NaN
1,3657,제주도%203657,https://m.place.naver.com/restaurant/37236930/...,NaN
2,가치,제주도%20가치,https://m.place.naver.com/restaurant/119657204...,NaN
3,곰막,제주도%20곰막,https://m.place.naver.com/restaurant/31204698/...,NaN
4,너븐,제주도%20너븐,https://m.place.naver.com/restaurant/127423932...,NaN


In [241]:
df.to_csv('naver_map_url2.csv')

### menu_list 홈페이지로 바로 들어가게 url 수정

In [242]:
df = pd.read_csv('./naver_map_url2.csv', index_col=0)
df.reset_index(drop=True, inplace=True)
df

,MCT_NM,naver_keyword,naver_map_url
0,젠,제주도%20젠,https://m.place.naver.com/restaurant/36333194/...
1,3657,제주도%203657,https://m.place.naver.com/restaurant/37236930/...
2,가치,제주도%20가치,https://m.place.naver.com/restaurant/119657204...
3,곰막,제주도%20곰막,https://m.place.naver.com/restaurant/31204698/...
4,너븐,제주도%20너븐,https://m.place.naver.com/restaurant/127423932...
5,당당,제주도%20당당,https://m.place.naver.com/restaurant/158916277...
6,도렐,제주도%20도렐,https://m.place.naver.com/restaurant/525595743...
7,먹돌,제주도%20먹돌,https://m.place.naver.com/restaurant/113173785...
8,빵귿,제주도%20빵귿,https://m.place.naver.com/restaurant/320930171...
9,신관,제주도%20신관,https://m.place.naver.com/restaurant/31761975/...


In [236]:
df.head()

,MCT_NM,naver_keyword,naver_map_url
0,젠,제주도%20젠,https://m.place.naver.com/restaurant/36333194/...
1,3657,제주도%203657,https://m.place.naver.com/restaurant/37236930/...
2,가치,제주도%20가치,https://m.place.naver.com/restaurant/119657204...
3,곰막,제주도%20곰막,https://m.place.naver.com/restaurant/31204698/...
4,너븐,제주도%20너븐,https://m.place.naver.com/restaurant/127423932...


메뉴 리스트들 구하기 
+ BeautifulSoup 사용 
+ Sqg65가 메뉴들 이름이 담긴 클래스 
+ 메뉴들 다 붙여진 걸로 추출한 다음 ,로 join

In [244]:
for i, url in enumerate(df['naver_map_url']):

    time.sleep(2)


    try:

        req=requests.get(url)
        menu_list=[]
        html = req.content.decode('utf-8','replace') 
        soup = BeautifulSoup(html, 'html.parser')
        menus=soup.find_all("span",attrs={"class":"Sqg65"})
        for menu in menus:
            menu_list.append(menu.get_text())
        
        val = ', '.join(menu_list)
        df.loc[int(i),'menu_list'] = val

     
#     리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다. 
        menu_list=[]
        #menu_list.append()
        df.loc[int(i),'menu_list']='no_menu'
       
    print(f'{i}')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [245]:
df.tail(80)

,MCT_NM,naver_keyword,naver_map_url,menu_list
328,제주김만복동문시장점,제주도%20제주김만복동문시장점,https://m.place.naver.com/restaurant/37609800/...,"만복이네 김밥, 통전복주먹밥, 숯불갈비주먹밥, 전복컵밥, 왕전복죽, 전복성게해물면,..."
329,제주약수터올레시장점,제주도%20제주약수터올레시장점,https://m.place.naver.com/restaurant/177724476...,"4종 샘플러 (1040cc), 무료시음서비스, 제주 수제맥주 340cc 잔, 6종 ..."
330,중문수두리보말칼국수,제주도%20중문수두리보말칼국수,https://m.place.naver.com/restaurant/129623529...,"수두리보말칼국수, 전복죽, 물만두, 제주막걸리, 우도땅콩막걸리"
331,칠돈가제주공항직영점,제주도%20칠돈가제주공항직영점,https://m.place.naver.com/restaurant/197728357...,"제주흑돼지 근고기 600g, 김치찌개, 칠돈가 근고기 600g, 냉면, 된장찌개, 공기밥"
332,케이에프씨제주시청점,제주도%20케이에프씨제주시청점,https://m.place.naver.com/restaurant/110741680...,"케이준후라이, 코울슬로, 에그타르트, 비스켓, 징거버거"
333,테디밸리골프앤리조트,제주도%20테디밸리골프앤리조트,https://m.place.naver.com/restaurant/38460408/...,
334,파리바게뜨한라병원점,제주도%20파리바게뜨한라병원점,https://m.place.naver.com/restaurant/115931982...,"아이스티, 에이드, 스무디, 딸기라떼, 팥빙수"
335,폴 바셋 제주용담DT점,제주도%20폴 바셋 제주용담DT점,https://m.place.naver.com/restaurant/172836308...,"룽고 (Standard), 카페라떼 (Standard), 상하목장 아이스크림"
336,안녕협재씨 제주협재점,제주도%20안녕협재씨 제주협재점,https://m.place.naver.com/restaurant/109490921...,"딱새우장비빔밥, 통전복내장비빔밥, 돌문어장비빔밥, 도마반판, 도마한판"
337,엔츠노래타운연동3호점,제주도%20엔츠노래타운연동3호점,https://m.place.naver.com/restaurant/133740623...,"소주, 맥주"


In [246]:
df.iloc[333,2]

'https://m.place.naver.com/restaurant/38460408/menu/list'

In [247]:
df.to_csv('naver_menu_crawling.csv')

In [178]:
df2=pd.read_csv('./naver_map_url.csv')
df2.head()

,Unnamed: 0,MCT_NM,naver_keyword,naver_map_url
0,0,젠,제주도%20젠,https://m.place.naver.com/restaurant/36333194/...
1,1,3657,제주도%203657,https://m.place.naver.com/restaurant/37236930/...
2,2,가치,제주도%20가치,https://m.place.naver.com/restaurant/119657204...
3,3,곰막,제주도%20곰막,https://m.place.naver.com/restaurant/31204698/...
4,4,너븐,제주도%20너븐,https://m.place.naver.com/restaurant/127423932...
